<a href="https://colab.research.google.com/github/SolisProcopioUriel/Modelaci-n-financiera/blob/main/ModeloBinomial_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import math
import ipywidgets as widgets
from IPython.display import display

def calcular_precio_opcion(tipo_opcion, tipo_contrato, S_0, K, r, sigma, T, n):
    dt = T / n
    u = math.exp(sigma * math.sqrt(dt))
    d = 1 / u
    p = (math.exp(r * dt) - d) / (u - d)

    # Construir el árbol de precios
    precios = [[0.0 for _ in range(i + 1)] for i in range(n + 1)]
    for i in range(n + 1):
        for j in range(i + 1):
            precios[i][j] = S_0 * (u ** j) * (d ** (i - j))

    # Imprimir el árbol binomial
    print("\nÁrbol Binomial de Precios:")
    for i in range(n + 1):
        espacios = " " * (n - i) * 8  # Ajusta el espacio para la visualización
        print(espacios, end="")
        for j in range(i + 1):
            print(f"{precios[i][j]:.4f}\t", end="")
        print()

    # Calcular los pagos finales según el tipo de opción y contrato
    pagos_finales = [0.0 for _ in range(n + 1)]
    for j in range(n + 1):
        if tipo_contrato == "CALL":
            pagos_finales[j] = max(0, precios[n][j] - K)
        elif tipo_contrato == "PUT":
            pagos_finales[j] = max(0, K - precios[n][j])

    # Retroceder a través del árbol para calcular el precio de la opción
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            beneficio_ejercicio = max(0, K - precios[i][j])  # Beneficio al ejercer la opción PUT
            valor_operacion = math.exp(-r * dt) * (p * pagos_finales[j + 1] + (1 - p) * pagos_finales[j])

            # Para el PUT Americano, considerar el ejercicio óptimo en cualquier momento
            if tipo_contrato == "PUT" and beneficio_ejercicio > valor_operacion:
                pagos_finales[j] = beneficio_ejercicio
            else:
                pagos_finales[j] = valor_operacion

    # El precio de la opción está en el nodo inicial
    return pagos_finales[0]

# Crear widgets
tipo_opcion_widget = widgets.Dropdown(options=['Europea', 'Americana'], value='Europea', description='Tipo de Opción:')
tipo_contrato_widget = widgets.Dropdown(options=['CALL', 'PUT'], value='CALL', description='Tipo de Contrato:')
S_0_widget = widgets.FloatText(value=100, description='S_0:')
K_widget = widgets.FloatText(value=100, description='K:')
r_widget = widgets.FloatText(value=0.05, description='r:')
sigma_widget = widgets.FloatText(value=0.2, description='σ:')
T_widget = widgets.FloatText(value=1, description='T:')
n_widget = widgets.IntText(value=100, description='n:')

# Crear botón y salida
calcular_button = widgets.Button(description='Calcular')
output = widgets.Output()

# Definir función de cálculo
def on_button_click(b):
    with output:
        tipo_opcion = tipo_opcion_widget.value
        tipo_contrato = tipo_contrato_widget.value
        S_0 = S_0_widget.value
        K = K_widget.value
        r = r_widget.value
        sigma = sigma_widget.value
        T = T_widget.value
        n = n_widget.value

        # Calcular y mostrar el precio de la opción
        precio_opcion = calcular_precio_opcion(tipo_opcion, tipo_contrato, S_0, K, r, sigma, T, n)
        print(f"\nEl precio de la opción {tipo_contrato} {tipo_opcion} es: {precio_opcion:.4f}")

# Asociar la función al botón
calcular_button.on_click(on_button_click)

# Mostrar widgets y salida
display(tipo_opcion_widget, tipo_contrato_widget, S_0_widget, K_widget, r_widget, sigma_widget, T_widget, n_widget, calcular_button, output)


Dropdown(description='Tipo de Opción:', options=('Europea', 'Americana'), value='Europea')

Dropdown(description='Tipo de Contrato:', options=('CALL', 'PUT'), value='CALL')

FloatText(value=100.0, description='S_0:')

FloatText(value=100.0, description='K:')

FloatText(value=0.05, description='r:')

FloatText(value=0.2, description='σ:')

FloatText(value=1.0, description='T:')

IntText(value=100, description='n:')

Button(description='Calcular', style=ButtonStyle())

Output()